In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

import requests
from PIL import Image
from io import BytesIO
import os
import csv

from transformers import ViTFeatureExtractor, ViTForImageClassification
import torch

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytr

In [18]:
from transformers import ViTForImageClassification, ViTFeatureExtractor
import torch
from PIL import Image
import os
import csv

# Initialize model and feature extractor
model = ViTForImageClassification.from_pretrained('nateraw/vit-age-classifier')
feature_extractor = ViTFeatureExtractor.from_pretrained('nateraw/vit-age-classifier')

# Directory mapping
image_directories = {
    "clean": '/Users/samin/Desktop/Classes/9.60/9.60-Project/datasets/clean_dataset'
}
# Adding filter directories
for i in range(1, 13):
    image_directories[f"filter {i}"] = f'/Users/samin/Desktop/Classes/9.60/9.60-Project/datasets/filter {i}'

def evaluate_model(image_dir, csv_file_path, dataset_name):
    predictions = []
    # Process each image
    for image_name in os.listdir(image_dir):
        image_path = os.path.join(image_dir, image_name)
        try:
            with Image.open(image_path) as im:
                inputs = feature_extractor(images=im, return_tensors="pt")
                outputs = model(**inputs)
                proba = torch.softmax(outputs.logits, dim=1)
                pred_class = proba.argmax(1).item()
                predictions.append((image_name, pred_class))
        except Exception as e:
            print(f"An error occurred with image {image_name}: {e}")

    # Read labels
    actual_labels = {}
    with open(csv_file_path, mode='r') as csvfile:
        csvreader = csv.DictReader(csvfile)
        for row in csvreader:
            actual_labels[row['image_name']] = int(row['class_label'])

    # Calculate accuracy
    correct_predictions = 0
    for image_name, pred_class in predictions:
        if image_name in actual_labels:
            actual_class = actual_labels[image_name]
            if pred_class == actual_class:
                correct_predictions += 1

    accuracy = correct_predictions / len(predictions) if predictions else 0
    print(f"Accuracy for {dataset_name}: {accuracy * 100:.2f}%")

# Evaluate model on datasets
for dataset_name, image_dir in image_directories.items():
    # Determine which labels file to use
    if "filter" in dataset_name:
        csv_file_path = '/Users/samin/Desktop/Classes/9.60/9.60-Project/datasets/labels 2.csv'
    else:
        csv_file_path = '/Users/samin/Desktop/Classes/9.60/9.60-Project/datasets/labels.csv'
    evaluate_model(image_dir, csv_file_path, dataset_name)


An error occurred with image .DS_Store: cannot identify image file '/Users/samin/Desktop/Classes/9.60/9.60-Project/datasets/clean_dataset/.DS_Store'
Accuracy for clean: 62.00%
An error occurred with image .DS_Store: cannot identify image file '/Users/samin/Desktop/Classes/9.60/9.60-Project/datasets/filter 1/.DS_Store'
Accuracy for filter 1: 58.00%
Accuracy for filter 2: 56.00%
Accuracy for filter 3: 60.00%
Accuracy for filter 4: 58.00%
Accuracy for filter 5: 58.00%
Accuracy for filter 6: 52.00%
Accuracy for filter 7: 58.00%
Accuracy for filter 8: 50.00%
Accuracy for filter 9: 52.00%
Accuracy for filter 10: 0.00%
Accuracy for filter 11: 0.00%
An error occurred with image .DS_Store: cannot identify image file '/Users/samin/Desktop/Classes/9.60/9.60-Project/datasets/filter 12/.DS_Store'
Accuracy for filter 12: 0.00%
